In [1]:
from matplotlib import pyplot as plt
import polars as pl
import seaborn as sns
import japanize_matplotlib

from common import *

In [2]:
N_COMPONENTS = 50
TEST_SIZE = 0.2
VAL_SIZE = 0.2
BAG_SIZE = 4

In [3]:
X_train, X_val, X_test, y_train, y_val, y_test = load_mnist_pca_train_test_val(
    n_components=N_COMPONENTS, test_size=TEST_SIZE, val_size=VAL_SIZE
)

Files already downloaded
Files already downloaded


In [4]:
X_train, y_train = rewrite_label_with_binary_setting(X_train, y_train)
X_val, y_val = rewrite_label_with_binary_setting(X_val, y_val)
X_test, y_test = rewrite_label_with_binary_setting(X_test, y_test)

In [5]:
# 1000個の正例を持つデータセットを作成する
X_train, y_train = rewrite_label_with_pu_setting(X_train, y_train, positive_size=1000)

In [6]:
def train_lgbm_with_custom_loss(X_train, X_valid, y_train, y_valid):

    """Train LightGBM with custom loss function."""
    train_data = lgb.Dataset(X_train, label=y_train, init_score=np.full_like(y_train, np.log(1.), dtype=float), free_raw_data=False)
    train_data._pu_label = y_train
    valid_data = lgb.Dataset(X_valid, label=y_valid, free_raw_data=False)

    params = {
        "objective": "custom",
        "metric": "custom",
        "verbose": -1,
        # 学習が不安定なので、小さめの値を設定します
        # 今回は単純な検証目的なので、ご容赦ください
        "learning_rate": 0.01,
        "num_boost_round": 200,
    }
    valid_accuracies = []

    def record_accuracies(p: lgb.Booster, train_data: lgb.Dataset, valid_data: lgb.Dataset):
        valid_pred = (p.predict(valid_data.data) > 0.).astype(int)
        valid_acc = accuracy_score(valid_data.label, valid_pred)
        valid_accuracies.append(valid_acc)
        print(valid_acc)
    
    gbm = lgb.train(
        params,
        train_data,
        valid_sets=[valid_data],
        fobj=pu_loss_objective,
        feval=binary_metric,
        callbacks=[lambda p: record_accuracies(p.model, train_data, valid_data)]
    )
    return gbm, valid_accuracies

In [7]:
gbm, valid_accuracies =train_lgbm_with_custom_loss(X_train, X_val, y_train, y_val)
y_pred = gbm.predict(X_val)
print("Accuracy: {}".format(accuracy_score(y_val, (y_pred > 0.).astype(int))))

/Users/ryoma.kobayashi/gits/techblog_git/wsl_w_lgbm/.venv/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


0.80375
0.8129166666666666
0.796
0.7951666666666667
0.813
0.8140833333333334
0.8193333333333334
0.8191666666666667
0.82025
0.8205
0.8219166666666666
0.8223333333333334
0.8226666666666667
0.8204166666666667
0.823
0.82325
0.8235
0.82425
0.826
0.8248333333333333
0.82525
0.8229166666666666
0.822
0.8239166666666666
0.8253333333333334
0.8269166666666666
0.8274166666666667
0.8285
0.8291666666666667
0.8279166666666666
0.8289166666666666
0.8294166666666667
0.8291666666666667
0.8294166666666667
0.82925
0.8294166666666667
0.8300833333333333
0.8298333333333333
0.8288333333333333
0.8288333333333333
0.8288333333333333
0.8280833333333333
0.8286666666666667
0.8300833333333333
0.8310833333333333
0.82925
0.83075
0.83175
0.8335
0.8335833333333333
0.834
0.83475
0.8361666666666666
0.8371666666666666
0.8394166666666667
0.8395
0.8411666666666666
0.84
0.84025
0.8406666666666667
0.84175
0.8413333333333334
0.84275
0.84475
0.84525
0.8459166666666667
0.8455
0.8466666666666667
0.8470833333333333
0.84775
0.84775
0.

In [8]:
np.save("pu_valid_accuracies.npy", valid_accuracies)